Train GCNN models

In [ ]:
# Loading modules

from __future__ import print_function, division

import numpy as np
import multiprocessing

from ase import io
from ase.db import connect

from tinnet.feature.voronoi import Voronoi
from tinnet.regression.regression import Regression

In [ ]:
# Setting variables

# Optimized hyperparameters
lr = 0.0026635818126661374
atom_fea_len = 202
n_conv = 10
h_fea_len = 93
n_h = 1

descriptor = Voronoi(max_num_nbr=12,
                     radius=8,
                     dmin=0,
                     step=0.2,
                     dict_atom_fea=None)

db = connect('../Database.db')

images = np.array([r.toatoms() for r in db.select()])

d_cen = np.array([r['data']['d_cen'] for r in db.select()], dtype=np.float32)
full_width = np.array([r['data']['full_width'] for r in db.select()], dtype=np.float32)

idx = np.arange(len(images))

idx_1 = idx[:-38]
idx_2 = idx[-38:] # Last 38 images are pure metals

num = int(len(idx_1)*1.00) # % of database for training (1.00 means 100%)
np.random.seed(12345)
np.random.shuffle(idx_1)

idx_1 = idx_1[0:num]

idx = np.sort(np.concatenate((idx_1,idx_2)))

images = [images[i] for i in idx]

d_cen = np.array([d_cen[i] for i in idx])
full_width = np.array([full_width[i] for i in idx])

features = multiprocessing.Pool().map(descriptor.feas, images)

final_ans_val_mae = np.zeros(10)
final_ans_val_mse = np.zeros(10)
final_ans_test_mae = np.zeros(10)
final_ans_test_mse = np.zeros(10)

idx_test = 9
idx_validation = 8

In [ ]:
# Setting up the model

model = Regression(features,
                   d_cen,
                   phys_model='gcnn_multitask',
                   optim_algorithm='AdamW',
                   weight_decay=0.0001,
                   idx_validation=idx_validation,
                   idx_test=idx_test,
                   lr=lr,
                   atom_fea_len=atom_fea_len,
                   n_conv=n_conv,
                   h_fea_len=h_fea_len,
                   n_h=n_h,
                   full_width=full_width,
                   batch_size=64)

In [ ]:
# Train the network

final_ans_val_mae[idx_test], final_ans_val_mse[idx_test],\
    final_ans_test_mae[idx_test], final_ans_test_mse[idx_test]\
        = model.train(25000)

np.savetxt('final_ans_val_MAE.txt', final_ans_val_mae)
np.savetxt('final_ans_val_MSE.txt', final_ans_val_mse)
np.savetxt('final_ans_test_MAE.txt', final_ans_test_mae)
np.savetxt('final_ans_test_MSE.txt', final_ans_test_mse)